<a href="https://colab.research.google.com/github/chw8207/pytorch_study/blob/main/%EC%8B%A0%EA%B2%BD%EB%A7%9D%EA%B8%B0%EC%B4%881.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 타입을 확인하는 함수

In [1]:
# 넘파이로 간단한 함수 구현하기
import numpy as np
from numpy import ndarray

def square(x: ndarray) -> ndarray :
  '''
  인자로 받은 ndarray 배열의 각 요솟값을 제곱한다.
  '''
  return np.power(x, 2)

def leaky_relu(x: ndarray) -> ndarray :
  '''
  ndarray 각 배열의 요소에 'Leaky ReLu함수를 적용한다.'
  '''
  return np.maximum(0.2*x, x)

### 도함수

In [2]:
# func: Callable[[ndarray], ndarray]
#       인자로 ndarray를 입력으로 받고, ndarray타입값을 반환하는 함수
from typing import Callable

def deriv(func: Callable[[ndarray], ndarray],
          input_: ndarray,
          delta: float = 0.001) -> ndarray :
    '''
    배열 input이 각 요소에 대해 함수 func의 도함수값 계산
    '''
    return (func(input_ + delta) - func(input_ - delta)) / (2 * delta)

### 합성함수

In [3]:
# 데이터 타입 정의
from typing import List

# ndarray를 인자로 받고 ndarray를 반환하는 함수
Array_Function = Callable[[ndarray], ndarray]

# Chain은 함수의 리스트다.
Chain = List[Array_Function]

In [4]:
Chain

typing.List[typing.Callable[[numpy.ndarray], numpy.ndarray]]

In [5]:
# 합성함수로 처리
def chain_length_2(chain: Chain, a: ndarray) -> ndarray :
  '''
  두 함수를 연쇄(chain)적으로 평가
  '''
  # 인자 chain의 길이는 2여야 함(2개의 함수 연결)
  assert len(chain) == 2

  f1 = chain[0]
  f2 = chain[1]

  return f2(f1(a))

### 합성함수의 도함수

In [7]:
def sigmoid(x: ndarray) -> ndarray :
  '''
  입력으로 받은 ndarray의 각 요소에 대한 sigmoid 함수값을 계산한다.
  '''
  return 1 / (1 + np.exp(-x))

In [8]:
# 연쇄법칙 구현
def chain_deriv_2(chain: Chain,
                  input_range: ndarray) -> ndarray :
    '''
    두 함수로 구성된 합성함수의 도함수를 계산하기 위해 연쇄법칙 사용
    (f2(f1(x))' = f2'(f1(x)) * f1'(x)
    '''
    assert len(chain) == 2

    # input_range는 1차원 ndarray여야 함
    assert input_range.ndim == 1
    f1 = chain[0]
    f2 = chain[1]

    # df1/dx
    f1_of_x = f1(input_range)

    # df1/du
    df1dx = deriv(f1, input_range)

    # df2/du(f1(x))
    df2du = deriv(f2, f1(input_range))

    # 각 점끼리 값을 곱함
    return df1dx * df2du